# Single Model Analysis

In [1]:
# Libraries

import warnings
warnings.filterwarnings('ignore')

# b) Data extraction
import os, base64
import sys
import xml.etree.ElementTree as ET
import rdflib
import re
from lxml import etree

# c) General
import copy
import difflib
import numpy as np
from numpyencoder import NumpyEncoder
import pandas as pd
import math
import operator as op
import ast
import re
import string
import csv
import xlsxwriter
import json
import ipywidgets as widgets
import ipyvuetify
from ipywidgets import interact, interact_manual, interactive, IntSlider, FloatSlider
from IPython.display import display, clear_output, Image, FileLink, HTML
from SPARQLWrapper import SPARQLWrapper, JSON
from PIL import Image
from scipy.optimize import curve_fit
from github import Github
import requests

# d) Plot
import matplotlib.pyplot as plt
from matplotlib import markers
import matplotlib.font_manager as font_manager
import matplotlib.colors
from pylab import rcParams
import matplotlib.image as mpimg



In [2]:
# Ontologies:
# Save the .csv files of each ontology from https://bioportal.bioontology.org/ontologies
FMA = pd.read_csv('./FMA.csv')
CHEBI = pd.read_csv('./CHEBI.csv')
OPB = pd.read_csv('./OPB.csv')
GO = pd.read_csv('./GO.csv')

# Extraction of IDs & Labels in the ontologies
fmaID = FMA['Class ID']
fmaLabel= FMA['Preferred Label']
chebiID = CHEBI['Class ID']
chebiLabel= CHEBI['Preferred Label']
opbID = OPB['Class ID']
opbLabel= OPB['Preferred Label']
goID = GO['Class ID']
goLabel= GO['Preferred Label']

In [3]:
def githubSearch(fileName):
    # github username
    github_username = "Niloofar-Sh"

    tokenFile = open('./github_access_token.txt','r')
    github_access_token = tokenFile.read()

    repository_name = "static_website_Jlab"
    folder_path = 'singleModelAnalysis'

    file_name = fileName+".cellml"  # The names of the files you want to search for

    # Set up the authentication headers for the GitHub API
    headers = {
        "Authorization": f"Bearer {github_access_token}"
    }

    # Get the contents of the folder
    contents_url = f"https://api.github.com/repos/{github_username}/{repository_name}/contents/{folder_path}"
    response = requests.get(contents_url, headers=headers)
    response_json = response.json()

    # Search for the file name among the list of files in the folder
    file_found = False
    for item in response_json:
        if item["type"] == "file" and item["name"] == file_name:
            file_found = True
            file_url = item["html_url"]
            break

    # Print the download URL of the file, if found
    if file_found:
        return file_url
    else:
        print(f"File '{file_name}' not found in folder '{folder_path}'")

In [4]:
def rdfMeaning(meaning,var,el):
    
    if 'OPB' in el:
        numCell=re.findall(r'\d+', el)
        for j in range(len(opbID)):
            numID=re.findall(r'\d+', opbID[j])
            if numCell == numID:
                meaning[var].append(opbLabel[j])
    if 'CHEBI' in el:
        numCell=re.findall(r'\d+', el)
        for j in range(len(chebiID)):
            numID=re.findall(r'\d+', chebiID[j])
            if numCell == numID:
                meaning[var].append(chebiLabel[j])
    if 'GO' in el:
        numCell=re.findall(r'\d+', el)
        for j in range(len(goID)):
            numID=re.findall(r'\d+', goID[j])
            if numCell == numID:
                meaning[var].append(goLabel[j])
    if 'FMA' in el:
        numCell=re.findall(r'\d+', el)
        for j in range(len(fmaID)):
            numID=re.findall(r'\d+', fmaID[j])
            if numCell == numID:
                meaning[var].append(fmaLabel[j])
    if ('OPB' not in el) and ('CHEBI' not in el) and('GO' not in el) and ('FMA' not in el):
        meaning[var].append(el)
                
    return meaning

In [5]:
# Retreiving the annotations from the CellML parameter files
def getAnnotations(add):
    parser = etree.XMLParser(recover=True)
    root = etree.parse(add, parser).getroot()
    rdfGraph = rdflib.Graph()
    for rdfElement in root.iter():
        if rdfElement.tag.endswith('RDF'):
            try:
                rdfGraph.parse(data = etree.tostring(rdfElement), format="application/rdf+xml")
            except:
                pass
            
    def getLeaves(sbj, graph):
        triples = list(graph.triples((sbj,None,None)))
        leaves = []
        if len(triples)>0:
            for s, p, o in triples:
                result =  getLeaves(o,graph)
                leaves += result
            return list(set(leaves))
        else:
            return [sbj]
        
    # Opens the CellML file and returns the list of variables names
    f = open(add,'r')
    text = f.read()
    root = ET.fromstring(text)

    rdfs = root.findall('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}RDF')

    List1=[]
    for child in rdfs:
        for grand in child:
            List1.append(grand.attrib.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}about'))
    
    List = list(dict.fromkeys(List1))
    if '#metaid0' in List:
        List.remove('#metaid0')
        
        
    triplesList={}
    for i in range(len(List)):
        sbj = rdflib.URIRef(List[i])
        triplesList[i]=getLeaves(sbj, rdfGraph)


    rdfs=[]
    
    for item in triplesList:
        d = []
        for k in range(len(triplesList[item])):
            if 'opb' in triplesList[item][k] or 'chebi' in triplesList[item][k] or 'fma' in triplesList[item][k] or 'go' in triplesList[item][k]: 
                lSplit= triplesList[item][k].split('/')
                d.append(lSplit[-1])
            else: # free-style descriptions!
                d.append(str(triplesList[item][k]))
    
        rdfs.append(d)

    for child in root:
        if child.tag == '{http://www.cellml.org/cellml/1.1#}'+'component':
            modelComponentName = child.attrib.get('name')
    
    
    listP = []
    for item in List:
        for x in item.split('.'):
            if '#'+modelComponentName == x:
                itemP = item.replace(x+'.', '')
                listP.append(itemP)
                

            
    return [listP,rdfs,root]

In [6]:
def valExtraction(root,List): 
    
    init=[]; variables=[]; els=[]
    
    for child in root:
        if child.tag == '{http://www.cellml.org/cellml/1.1#}'+'component':
            modelComponentName = child.attrib.get('name')
            
    components = root.findall('{http://www.cellml.org/cellml/1.1#}component')
    
    for comp in components:
        variables.append(comp.findall('{http://www.cellml.org/cellml/1.1#}variable'))
        
    for element in List:  
        for var in variables:
            for v in var:   
                if modelComponentName+'.'+element == v.attrib['{http://www.cellml.org/metadata/1.0#}id']:
                    if 'initial_value' in v.attrib: # if any initial value exists take it
                        init.append(v.attrib['initial_value'])    
                    else:
                        init.append('None')
    return init

In [99]:
 def textFileGen(speciesNoDuplicate,kfkr_estimated,parametersData,meaning,reaction_reactants,V,N,reaction_products):

#  def textFileGen(speciesNoDuplicate,parametersData,speciesConstants,reaction_reactants,V,N,reaction_products):
    # Create a text file (TEXT)

    unitsImport = ['def model composed_model as',
                   '\tdef import using "units_BG.cellml" for',
                    '\t\tunit concentrationUnit using unit concentrationUnit;',
                   '\t\tunit speciesConstantUnit using unit speciesConstantUnit;',
                   '\t\tunit fluxUnit using unit fluxUnit;',
                    '\tenddef;',
                  '\tdef comp main as']
    



    with open('GFG.txt', 'w') as fp:
        pass    
    with open('GFG.txt', 'a') as file:
        for line in unitsImport:
            file.write(line+'\n')


    with open('GFG.txt', 'a') as file:
        file.write('\t\tvar t: second {init: 0};'+'\n')

    cellmlRef=[]        

    q = ['q_{0}'.format(x) for x in range(1000)]
    k=0
    checkDup=[]
    for species,i in zip(speciesNoDuplicate['speciesNoDuplicate'],range(len(speciesNoDuplicate['speciesNoDuplicate']))):
            for transporterName,annotation in parametersData.items():
                for value in annotation:
                    if all(el in value[1] for el in species):
                        flag=0; 
                        for j in range(len(checkDup)):
                            if all(x in checkDup[j] for x in value[1]):
                                flag=1

                        if flag==0:
                            checkDup.append(value[1])
                            cellmlRef.append((value[1],q[k],'concentrationUnit'))
                            with open('GFG.txt', 'a') as file:
                                file.write('\t\tvar ' +q[k]+': '+'concentrationUnit '+'{'+'init: '+value[2]+'}'+';'+'\n')
                                k+=1


    v = ['v_{0}'.format(x) for x in range(1000)]
    k=0
    for key in reaction_reactants:
        for flux in reaction_reactants[key]:
            cellmlRef.append((flux,v[k],'fluxUnit'))
            with open('GFG.txt', 'a') as file:
                file.write('\t\tvar '+v[k]+': '+'fluxUnit '+';'+'\n')
                k+=1


    kf = ['kf_{0}'.format(x) for x in range(1000)]
    kr = ['kr_{0}'.format(x) for x in range(1000)]
    k=0
    for key in V:
        for flux,i in zip(V[key],range(len(V[key]))): 
            for value in parametersData[key]:
                    if flux == value[0]:
                        cellmlRef.append((value[1],kf[k],'fluxUnit'))

                        with open('GFG.txt', 'a') as file:
                            file.write('\t\tvar '+ kf[k]+': '+'fluxUnit '+'{'+'init: '+str(kfkr_estimated[key][flux]['kf'])+'}'+';'+'\n')
                            file.write('\t\tvar '+ kr[k]+': '+'fluxUnit '+'{'+'init: '+str(kfkr_estimated[key][flux]['kr'])+'}'+';'+'\n')
                            k+=1


    # Generating the ODEs       
    with open('GFG.txt', 'a') as file:
        file.write('\n')

    for i in range(np.shape(N[singleSelection.value])[0]):
        for annotation,ID,unit in cellmlRef:
            if all(ele in annotation for ele in speciesNoDuplicate['speciesNoDuplicate'][i]):
                with open('GFG.txt', 'a') as file:
                    file.write('\t\tode('+ID+',t)=')
                for j in range(np.shape(N[singleSelection.value])[1]):  
                    if N[singleSelection.value][i][j] != 0:
                        
                        if N[singleSelection.value][i][j] > 0:
                            for annotation2,ID2,unit2 in cellmlRef:
                                
                                if  all(el in annotation2 for el in V[singleSelection.value][j]):
                                    with open('GFG.txt', 'a') as file:
                                        file.write('+'+str(N[singleSelection.value][i][j])+'{dimensionless}*'+ID2)

                    if N[singleSelection.value][i][j] < 0:
                        for annotation2,ID2,unit2 in cellmlRef:
                            if all(el in annotation2 for el in V[singleSelection.value][j]):
                                with open('GFG.txt', 'a') as file:
                                    file.write(str(N[singleSelection.value][i][j])+'{dimensionless}*'+ID2)

        with open('GFG.txt', 'a') as file:
            file.write(';\n')


    # Generating the reaction rates
    with open('GFG.txt', 'a') as file:
        file.write('\n')

    for flux,i in zip(V[singleSelection.value],range(len(V[singleSelection.value]))):
        for annotation1,ID1,unit1 in cellmlRef:
            if all(el in annotation1 for el in flux):
                with open('GFG.txt', 'a') as file:
                    file.write('\t\t '+ ID1+'= ') 
                    
        with open('GFG.txt', 'a') as file:
            file.write('(')
        
        if singleSelection.value=='SGLT1' and 'sodium(1+)' in meaning[flux+'_'+singleSelection.value]:
            z_Na=1; F=96485; T=310; R=8.314; V_na=-0.046;
            membraneContrib = math.exp((z_Na*F*V_na)/(R*T))
            with open('GFG.txt', 'a') as file:
                file.write(str(membraneContrib)+'{dimensionless}'+'*')

        length = 0    
        for key in reaction_reactants:
            for stoi,reactant in reaction_reactants[key][flux]:
                length+=1
                with open('GFG.txt', 'a') as file:
                    file.write('kf_'+str(i)+'*') 
                for annotation,ID,unit in cellmlRef:
                    if all(el in reactant for el in annotation):
                        with open('GFG.txt', 'a') as file:
                            file.write('pow('+ID+','+str(stoi)+'{dimensionless}'+')') 

                if length < len(reaction_reactants[key][flux]):
                    with open('GFG.txt', 'a') as file:
                        file.write(' * ')
                        


            with open('GFG.txt', 'a') as file:
                file.write(' - ')

        length = 0 
        for key in reaction_products:
            for stoi,product in reaction_products[key][flux]:
                length+=1
                with open('GFG.txt', 'a') as file:
                    file.write('kr_'+str(i)+'*') 
                for annotation,ID,unit in cellmlRef:
                    if all(el in product for el in annotation):
                        with open('GFG.txt', 'a') as file:
                            file.write('pow('+ID+','+str(stoi)+'{dimensionless}'+')') 

                if length < len(reaction_products[key][flux]):
                    with open('GFG.txt', 'a') as file:
                        file.write(' * ')




        with open('GFG.txt', 'a') as file:
            file.write(');\n')





    with open('GFG.txt', 'a') as file:
        file.write('\tenddef;'+'\n'+'enddef;')
        
    return cellmlRef

In [8]:
def pmrSearching(name):
    sparqlendpoint = 'https://models.physiomeproject.org/pmr2_virtuoso_search'
    sparql = SPARQLWrapper(sparqlendpoint)

    def search_entity(terms):
        query = """SELECT ?graph ?Model_entity WHERE {{ GRAPH ?graph {{ ?Model_entity ?p ?o FILTER REGEX(LCASE(STR(?Model_entity)), '{terms}')}}}}""".format(terms=terms)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        graphs = sparql.query().convert()
        return graphs

    def search_model(terms):
        terms = terms.lower()
        entities = search_entity(terms)
        model = set()
        for entity in entities['results']['bindings']:
            workspace = entity['graph']['value']
            cellml = entity['Model_entity']['value'].split('#')[0]
            if not cellml.startswith('http') and terms in cellml.lower():
                model.update([workspace+'/rawfile/HEAD/'+cellml])
        return list(model)

    pmrModel = search_model(name)

    return pmrModel

In [107]:
def singleBGmodelBuilder(parametersData,inUse,reaction_reactants,reaction_products):

    V = []
    for key in reaction_reactants:
        for flux in reaction_reactants[key]:
            V.append(flux)

    Species = []
    for transporterName in parametersData:
        for entity in parametersData[transporterName]:
            if ('OPB_00340' in entity[1] and
               'OPB_00592' not in entity[1] and
               'OPB_01315' not in entity[1]):
                X = copy.deepcopy(entity[1])
                X.append(transporterName)
                Species.append(X)

    speciesNoDuplicate = []
    for s in Species:
        for transporterName in inUse:
            if transporterName in s:
                ss = copy.deepcopy(s)
                ss.remove(transporterName)
                if ss not in speciesNoDuplicate:
                    speciesNoDuplicate.append(ss)
  
    M=np.zeros((len(Species),len(V)))
    
    for s in range(len(Species)):
        for key in reaction_products:
            for key2,i in zip(reaction_products[key],range(len(reaction_products[key]))):
                for stoi,val  in reaction_products[key][key2]:
                    if all(el in val+[key] for el in Species[s]):
                        M[s][i] = float(stoi)
        for key in reaction_reactants:
            for key2,i in zip(reaction_reactants[key],range(len(reaction_reactants[key]))):
                for stoi,val in reaction_reactants[key][key2]:
                    if all(el in val+[key] for el in Species[s]):
                        M[s][i] = -float(stoi)
    
    N=np.zeros((len(speciesNoDuplicate),len(V)))
    for v,i in zip(V,range(len(V))):
        for s1 in range(len(speciesNoDuplicate)):
            for s2 in range(len(Species)):
                if all(el in Species[s2] for el in speciesNoDuplicate[s1]):
                    N[s1][i] = M[s2][i]

    return [N,V,speciesNoDuplicate]


In [10]:
def pmrRDFsearch(terms):
    
    sparqlendpoint = 'https://models.physiomeproject.org/pmr2_virtuoso_search'
    sparql = SPARQLWrapper(sparqlendpoint)

    sparql_tmpl = '''SELECT DISTINCT ?g ?v WHERE {{ GRAPH ?g {{
    ?v ?p1 [?p2 ?b; ?p3 [?p4 ?d; ?p5 [?p6 ?f]]] .
    FILTER (regex(str(?b),"{opb}") && regex(str(?d),"{chebi}") && regex(str(?f),"{fma}"))
    }} }}'''

    for c in terms:
        if c.startswith('CHEBI'):
            chebi = c
        elif c.startswith('OPB'):
            opb = c
        elif c.startswith('FMA'):
            fma = c
    query = sparql_tmpl.format(opb=opb, chebi=chebi, fma=fma)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    rst =  sparql.query().convert()
    if len(rst['results']['bindings']) > 0:
        cellmls = {}
        for r in rst['results']['bindings']:
            cellml_file, cellml_var = r['v']['value'].split('#')
            cellml_path = r['g']['value'] + '/rawfile/HEAD/' + cellml_file
            if cellml_path not in cellmls:
                cellmls[cellml_path] = []
            cellmls[cellml_path] += [cellml_var]
        return cellmls
    return {}


In [12]:
# Here we see which group of transporters/channels belong to which template
templateGroups = {'type1':['GLUT2', 'X1', 'X2', 'X3', '...'],
                  'type2':['SGLT1', 'Y1', 'Y2', 'Y3', '...']}

## Section 1: Model selection

In [67]:
# Model Selection and BG structure demonstration
singleSelection = widgets.Dropdown(
            options=['SGLT1', 'GLUT2', 'X1', 'X2', 'X3', 'SGLT2', 'Y1', 'Y2', 'Y3', '...'],
            description='Select model:',
    style={'description_width': 'initial'},
            disabled=False
        )
button_load = widgets.Button(
                description='Load model',
                tooltip='Load',
                style={'description_width': 'initial'}
            )
output1 = widgets.Output(layout={'border': '1px solid black'})

def button_load_clicked(event):
    with output1:
        clear_output()
        channelSelected = [singleSelection.value]
        inUse={}
        for selected in channelSelected:
            for key in templateGroups.keys():
                for transporterName,i in zip(templateGroups[key],range(len(templateGroups[key]))):
                    if selected == transporterName:
                        inUse[selected]=[]
                        inUse[transporterName].append(key)
                        
        pmrModel = pmrSearching(singleSelection.value)
        giturl = githubSearch(singleSelection.value)
        
#         print(f"The template is <{inUse[singleSelection.value][0]}> with the link to the original model: {pmrModel}")
        print(f"The original model <{singleSelection.value}> is deposited on: {giturl}")

        # Create an Output widget
        out = widgets.Output()

        # Create a button widget
        button = widgets.Button(description='Show the structure')

        # Define the path of the folder containing the images you want to display
        folder_path = './figures'

        # Define a function to display the images when the button is clicked
        def display_images(button_click):
            with out:
                out.clear_output()
                for file in os.listdir(folder_path):
                    if file.strip('.jpg') == inUse[singleSelection.value][0] or \
                    file.strip('.JPG') == inUse[singleSelection.value][0] or \
                    file.strip('.JPEG') == inUse[singleSelection.value][0] or \
                    file.strip('.jpeg') == inUse[singleSelection.value][0]: 
                        file_path = os.path.join(folder_path, file)
                        img = Image.open(file_path)
                        new_image = img.resize((600, 400))
                        text_0 = widgets.HTML(value="<h5><b>Bond graph structure of {}:<b><h5>".format(file.strip('.jpg')))
                        display(text_0,new_image)

        # Attach the function to the button's on_click event
        button.on_click(display_images)

        # Display the widgets
        display(button)
        display(out)
        
button_load.on_click(button_load_clicked)
vbox_result = widgets.VBox([button_load, output1])   

vbox_text = widgets.VBox([singleSelection, vbox_result])
page1 = widgets.HBox([vbox_text])

display(page1)

## Section 2: The Steady-states (SS)

In [92]:
button_SS = widgets.Button(
                description='Show SS',
                tooltip='Description',
                style={'description_width': 'initial'},
                layout={'width': 'auto'}
            )
output2 = widgets.Output(layout={'border': '1px solid black'})

def button_SS_clicked(event):
    with output2:
        clear_output()
        
        channelSelected = [singleSelection.value]
        inUse={}
        for selected in channelSelected:
            for key in templateGroups.keys():
                for transporterName,i in zip(templateGroups[key],range(len(templateGroups[key]))):
                    if selected == transporterName:
                        inUse[selected]=[]
                        inUse[transporterName].append(key)
        
        
        
        annotations={}
        for transporterName in inUse:
            
            addressData = './{0}.csv'.format(transporterName+'_Data')
            Data = pd.read_csv(addressData)
            df_Data = pd.DataFrame(Data)
            
            addressP = './{0}.cellml'.format(singleSelection.value)
            [List,rdfs,root] = getAnnotations(addressP)
            initialVals = valExtraction(root,List)
            annotations[transporterName] = []
            annotations[transporterName] = [List,rdfs,initialVals]
            
        # Combining the model info in one dictionary ==> parametersData
        parametersData = {}
        for key in annotations:
            parametersData[key] = []
            for j in range(len(annotations[key][0])):
                parametersData[key].append([annotations[key][0][j],annotations[key][1][j],annotations[key][2][j]])
        
        # Save the values in .json file
        json_object = json.dumps(parametersData, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False,
              cls=NumpyEncoder)
        with open("parametersData.json", "w") as outfile:
            outfile.write(json_object)
            
            
        SS = {}  
        meaning = {} 
        for key,values in parametersData.items():
            for value in values:
                for item in df_Data.columns:
                    if key+' | '+value[0] in item.split(' ('):
                        SS[value[0]+'_'+key] = df_Data[item].iloc[-1]  
                        if type(value[1])==list:
                            meaning[value[0]+'_'+key]=[]
                            for el in value[1]:
                                meaning = rdfMeaning(meaning,value[0]+'_'+key,el)
                                
        # Save the SS values in SS.json file
        json_object = json.dumps(SS, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False,
              cls=NumpyEncoder)
        with open("SS.json", "w") as outfile:
            outfile.write(json_object)
            
        # Save the meaning values in meaning.json file
        json_object = json.dumps(meaning, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False,
              cls=NumpyEncoder)
        with open("meaning.json", "w") as outfile:
            outfile.write(json_object)
            
            
        
            
        SSreaction={}
        reaction_reactants = {}
        reaction_products = {}

        for key,vals in parametersData.items():
            reaction_reactants[key]={}
            reaction_products[key]={}
            if key == 'SGLT1':
                for val in vals:
                    if 'OPB_00592' in val[1]:   # reaction finding
                        SSreaction[val[0]] = SS[val[0]+'_'+key]
                        reactionName = val[0]
                        singleSpeciesRdf=copy.deepcopy(val[1])  # reaction to what species?
                        singleSpeciesRdf.remove('OPB_00340')
                        singleSpeciesRdf.remove('OPB_00592')
                        reaction_reactants[key][reactionName]=[]
                        reaction_products[key][reactionName]=[]

                        for val0 in parametersData[key]:
                            if ('FMA:70022' in val0[1] and # being reactant in SGLT1
                                'OPB_00340' in val0[1] and # concentration Yes
                                'OPB_00592' not in val0[1] and # reaction rate No
                                'OPB_01315' not in val0[1] and # molecule No
                                singleSpeciesRdf[0] in val0[1]):
                                composedSpeciesRdf = copy.deepcopy(val0[1])                            
                        for val1 in parametersData[key]:
                            if (all(el in composedSpeciesRdf for el in val1[1]) and 
                                'OPB_00592' not in val1[1]):
                                stoi=1
                                for val2 in parametersData[key]:
                                    if ('OPB_00340' in val2[1] and 
                                    'OPB_00592' not in val2[1] and 
                                    singleSpeciesRdf[0] in val2[1] and
                                    'OPB_01315' in val2[1]):
                                        stoi= copy.deepcopy(float(val2[2]))
                                reaction_reactants[key][reactionName].append((stoi,composedSpeciesRdf))


                        for val0 in parametersData[key]:
                            if ('FMA:260691' in val0[1] and # being product in SGLT1
                                'OPB_00340' in val0[1] and # concentration Yes
                                'OPB_00592' not in val0[1] and # reaction rate No
                                'OPB_01315' not in val0[1] and # molecule No
                                singleSpeciesRdf[0] in val0[1]):
                                composedSpeciesRdf = copy.deepcopy(val0[1])
                        for val1 in parametersData[key]:
                            if (all(el in composedSpeciesRdf for el in val1[1]) and 
                                'OPB_00592' not in val1[1]):
                                stoi=1
                                for val2 in parametersData[key]:
                                    if ('OPB_00340' in val2[1] and 
                                    'OPB_00592' not in val2[1] and 
                                    singleSpeciesRdf[0] in val2[1] and
                                    'OPB_01315' in val2[1]):
                                        stoi= copy.deepcopy(float(val2[2]))
                                reaction_products[key][reactionName].append((stoi,composedSpeciesRdf))

            if key == 'GLUT2':
                for val in vals:
                    if 'OPB_00592' in val[1]:   # reaction finding
                        SSreaction[val[0]] = SS[val[0]+'_'+key]
                        reactionName = val[0]
                        singleSpeciesRdf=copy.deepcopy(val[1])  # reaction to what species?
                        singleSpeciesRdf.remove('OPB_00340')
                        singleSpeciesRdf.remove('OPB_00592')
                        reaction_reactants[key][reactionName]=[]
                        reaction_products[key][reactionName]=[]
                        
                        for val0 in parametersData[key]:
                            if ('FMA:226050' in val0[1] and # being reactant in GLUT2
                                'OPB_00340' in val0[1] and # concentration Yes
                                'OPB_00592' not in val0[1] and # reaction rate No
                                'OPB_01315' not in val0[1] and # molecule No
                                singleSpeciesRdf[0] in val0[1]):
                                composedSpeciesRdf = copy.deepcopy(val0[1])                            
                        for val1 in parametersData[key]:
                            if (all(el in composedSpeciesRdf for el in val1[1]) and 
                                'OPB_00592' not in val1[1]):
                                stoi=1
                                for val2 in parametersData[key]:
                                    if ('OPB_00340' in val2[1] and 
                                    'OPB_00592' not in val2[1] and 
                                    singleSpeciesRdf[0] in val2[1] and
                                    'OPB_01315' in val2[1]):
                                        stoi= copy.deepcopy(float(val2[2]))
                                reaction_reactants[key][reactionName].append((stoi,composedSpeciesRdf))


                        for val0 in parametersData[key]:
                            if ('FMA:70022' in val0[1] and # being product in GLUT2
                                'OPB_00340' in val0[1] and # concentration Yes
                                'OPB_00592' not in val0[1] and # reaction rate No
                                'OPB_01315' not in val0[1] and # molecule No
                                singleSpeciesRdf[0] in val0[1]):
                                composedSpeciesRdf = copy.deepcopy(val0[1])
                        for val1 in parametersData[key]:
                            if (all(el in composedSpeciesRdf for el in val1[1]) and 
                                'OPB_00592' not in val1[1]):
                                stoi=1
                                for val2 in parametersData[key]:
                                    if ('OPB_00340' in val2[1] and 
                                    'OPB_00592' not in val2[1] and 
                                    singleSpeciesRdf[0] in val2[1] and
                                    'OPB_01315' in val2[1]):
                                        stoi= copy.deepcopy(float(val2[2]))
                                reaction_products[key][reactionName].append((stoi,composedSpeciesRdf))
                    
         
        # Save the values in .json file
        json_object = json.dumps(reaction_reactants, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False,
              cls=NumpyEncoder)
        with open("reaction_reactants.json", "w") as outfile:
            outfile.write(json_object)
            
        json_object = json.dumps(reaction_products, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False,
              cls=NumpyEncoder)
        with open("reaction_products.json", "w") as outfile:
            outfile.write(json_object)     
            
            
        [N,V,speciesNoDuplicate] = singleBGmodelBuilder(parametersData,inUse,reaction_reactants,reaction_products)      
        
        
        # Save the values in .json file
        N = {singleSelection.value:N}
        V = {singleSelection.value:V}
        json_object = json.dumps(N, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False,
              cls=NumpyEncoder)
        with open("N.json", "w") as outfile:
            outfile.write(json_object)
            
        json_object = json.dumps(V, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False,
              cls=NumpyEncoder)
        with open("V.json", "w") as outfile:
            outfile.write(json_object)
            
        S = {'speciesNoDuplicate': speciesNoDuplicate}
        json_object = json.dumps(S, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False,
              cls=NumpyEncoder)
        with open("speciesNoDuplicate.json", "w") as outfile:
            outfile.write(json_object)
            

        
            


        text_0 = widgets.HTML(value="<h5><b>Your selected protein {} has the following SS values:<b><h5>".format(singleSelection.value))
        display(text_0)
        for key,val in meaning.items():
            text_1 = widgets.HTML(value="<h5><b>{}:<b> {}<h5> <h5>{}<h5>".format(key,SS[key],val))
            display(text_1)
        
                
button_SS.on_click(button_SS_clicked)
vbox_result = widgets.VBox([button_SS, output2]) 

page2 = widgets.HBox([vbox_result])

display(page2)


## Section 3: The kinetic values

In [105]:
button_kfkr = widgets.Button(
                description='Estimate the kinetic parameters',
                tooltip='Description',
                style={'description_width': 'initial'},
                layout={'width': 'auto'}
            )
output3 = widgets.Output(layout={'border': '1px solid black'})

def button_kfkr_clicked(event):
    with output3:
        clear_output()


        f = open("SS.json")
        SS = json.load(f)
        f = open("N.json")
        N = json.load(f)
        f = open("V.json")
        V = json.load(f)
        f = open("reaction_reactants.json")
        reaction_reactants = json.load(f)
        f = open("reaction_products.json")
        reaction_products = json.load(f)
        f = open("meaning.json")
        meaning = json.load(f)
        f = open("parametersData.json")
        parametersData = json.load(f)
                
        
        reactionRates = {}
        
        for key in V:
            reactionRates[key]={}
            for reaction in V[key]:

                reactantsSpecies=[]
                productsSpecies=[]


                solve=tuple()

                for stoichiometry,reactant in reaction_reactants[key][reaction]:
                    reactantsSpecies.append(reactant)       
                    for val in parametersData[key]:
                        if all(el in val[1] for el in reactant):
                            solve=solve+(pow(SS[val[0]+'_'+key] ,stoichiometry),) 


                for stoichiometry,product in reaction_products[key][reaction]:
                    productsSpecies.append(product)        
                    for val in parametersData[key]:
                        if all(el in val[1] for el in product):
                            solve=solve+(pow(SS[val[0]+'_'+key] ,stoichiometry),) 


                def Na_VG(X, kf, kr):
                    for x in range(0,len(reactantsSpecies)):

                        reactants = 1 * X[x]            
                    for x in range(len(reactantsSpecies),len(reactantsSpecies)+len(productsSpecies)):
                        products = 1 * X[x]

                    z_Na=1; F=96485; T=310; R=8.314; V_na=-0.046

                    v = kf*1e6*reactants*math.exp((z_Na*F*V_na)/(R*T)) - kr*1e6*products
                    # 1e6 added to all concentrations & rate because the values were too small
                    # just like including compartments in SBML models
                    return v

                def func(X, kf, kr):

                    for x in range(0,len(reactantsSpecies)):
                        reactants = 1 * X[x]      
                    for x in range(len(reactantsSpecies),len(reactantsSpecies)+len(productsSpecies)):
                        products = 1 * X[x]

                    v = kf*1e6*reactants - kr*1e6*products
                    # 1e6 added to all concentrations & rate because the values were too small
                    # just like including compartments in SBML models
                    return v
                
                def diffusion(X, kfr):

                    for x in range(0,len(reactantsSpecies)):
                        reactants = 1 * X[x]      
                    for x in range(len(reactantsSpecies),len(reactantsSpecies)+len(productsSpecies)):
                        products = 1 * X[x]

                    v = kfr*1e6*reactants - kfr*1e6*products
                    # 1e6 added to all concentrations & rate because the values were too small
                    # just like including compartments in SBML models
                    return v

                

                if key=='SGLT1' and 'sodium(1+)' in meaning[reaction+'_'+key]:
                    bounds=[[0,0],[np.inf,np.inf]]
                    popt, pcov = curve_fit(Na_VG,solve,1e6*SS[reaction+'_'+key], maxfev=3000000, bounds=bounds)
                    reactionRates[key][reaction]={}
                    reactionRates[key][reaction]['kf'] = popt[0] 
                    reactionRates[key][reaction]['kr'] = popt[1]
                if key=='SGLT1' and 'sodium(1+)' not in meaning[reaction+'_'+key]:
                    bounds=[[0,0],[np.inf,np.inf]]
                    popt, pcov = curve_fit(func,solve,1e6*SS[reaction+'_'+key], maxfev=3000000, bounds=bounds)
                    reactionRates[key][reaction]={}
                    reactionRates[key][reaction]['kf'] = popt[0] 
                    reactionRates[key][reaction]['kr'] = popt[1]
                if key=='GLUT2':
                    bounds=[[0],[np.inf]]
                    popt, pcov = curve_fit(diffusion,solve,1e6*SS[reaction+'_'+key], maxfev=3000000, bounds=bounds)
                    reactionRates[key][reaction]={}
                    reactionRates[key][reaction]['kf'] = popt[0] 
                    reactionRates[key][reaction]['kr'] = popt[0]

                print(reaction,':',reactionRates[key][reaction])


        text_0 = widgets.HTML(value="<h5><b>where:<b><h5>")
        display(text_0)
        for key,val in V.items():
            for reaction in val:
                text_1 = widgets.HTML(value="<h5><b>{}:<b><h5> <h5>{}<h5>".format(reaction,meaning[reaction+'_'+key]))
                display(text_1)


         # Create the export button widget
        button_export = widgets.Button(description='Export the values')
        output4 = widgets.Output()
        # Define a function to handle the button click event
        def export_values(button_export):
            with output4:
                clear_output()
                # Save the values in .json file
                json_object = json.dumps(reactionRates, indent=4, sort_keys=True,
                      separators=(', ', ': '), ensure_ascii=False,
                      cls=NumpyEncoder)
                with open("kfkr_estimated.json", "w") as outfile:
                    outfile.write(json_object)
                text_2 = widgets.HTML(value="<h5>Estimated values for kf and kr saved to kfkr_estimated.json<h5>")
                display(text_2)

        # Attach the save_number function to the button click event
        button_export.on_click(export_values)
        # Display the widgets
        vbox_result = widgets.VBox([button_export, output4]) 
        page4 = widgets.HBox([vbox_result])
        display(page4)

      
        
button_kfkr.on_click(button_kfkr_clicked)
vbox_result = widgets.VBox([button_kfkr, output3]) 

page3 = widgets.HBox([vbox_result])

display(page3)

## Section 4: BG model generation

In [106]:
button_cellml = widgets.Button(
                description='Generate code',
                tooltip='Description',
                style={'description_width': 'initial'}
            )
output5 = widgets.Output(layout={'border': '1px solid black'})

def on_button_clicked5(event):
    with output5:
        f = open("N.json")
        N = json.load(f)
        f = open("V.json")
        V = json.load(f)
        f = open("speciesNoDuplicate.json")
        speciesNoDuplicate = json.load(f)
        f = open("reaction_reactants.json")
        reaction_reactants = json.load(f)
        f = open("reaction_products.json")
        reaction_products = json.load(f)
        f = open("kfkr_estimated.json")
        kfkr_estimated = json.load(f)
        f = open("parametersData.json")
        parametersData = json.load(f)
        f = open("meaning.json")
        meaning = json.load(f)


        cellmlRef = textFileGen(speciesNoDuplicate,kfkr_estimated,parametersData,meaning,reaction_reactants,V,N,reaction_products)
        text_0 = widgets.HTML(value="<h3>The CellML code for the bond graph form of {} at Steady State is:</h3>".format(singleSelection.value))
        vbox_text = widgets.VBox([text_0])
        display(vbox_text)
        with open('GFG.txt') as f:
            contents = f.read()
        print(contents)

button_cellml.on_click(on_button_clicked5)
vbox_result = widgets.VBox([button_cellml, output5]) 
display(vbox_result)

